# Setup 

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords

In [5]:
df = pd.read_csv('https://github.com/Adkurrr/Sentiment-Analysis-with-Aspect-Based-Approach-on-Spotify-App-with-SVM/blob/main/Dataset/reviews.csv?raw=true', delimiter=',')
df.head()

,Time_submitted,Review,Rating,Total_thumbsup,Reply
0,2022-07-09 15:00:00,"Great music service, the audio is high quality...",5,2,NaN
1,2022-07-09 14:21:22,Please ignore previous negative rating. This a...,5,1,NaN
2,2022-07-09 13:27:32,"This pop-up ""Get the best Spotify experience o...",4,0,NaN
3,2022-07-09 13:26:45,Really buggy and terrible to use as of recently,1,1,NaN
4,2022-07-09 13:20:49,Dear Spotify why do I get songs that I didn't ...,1,1,NaN


# Praproses Data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61594 entries, 0 to 61593
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Time_submitted  61594 non-null  object
 1   Review          61594 non-null  object
 2   Rating          61594 non-null  int64 
 3   Total_thumbsup  61594 non-null  int64 
 4   Reply           216 non-null    object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


### Handling missing value

In [8]:
df.isnull().sum()

Time_submitted        0
Review                0
Rating                0
Total_thumbsup        0
Reply             61378
dtype: int64

In [ ]:
df.drop('Reply', axis=1, inplace=True)

### Handing duplicated data

In [15]:
df.duplicated().sum()

0

### Lowercase, clear stop words and clear punctuations

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
def clean(review):
    review = review.lower()
    review = re.sub('[^a-z A-Z 0-9-]+', '', review)
    review = " ".join([word for word in review.split() if word not in stopwords.words('english')])

    return review

In [24]:
df['Review'] = df['Review'].apply(clean)
df.head(10)

,Time_submitted,Review,Rating,Total_thumbsup
0,2022-07-09 15:00:00,great music service audio high quality app eas...,5,2
1,2022-07-09 14:21:22,please ignore previous negative rating app sup...,5,1
2,2022-07-09 13:27:32,pop-up get best spotify experience android 12 ...,4,0
3,2022-07-09 13:26:45,really buggy terrible use recently,1,1
4,2022-07-09 13:20:49,dear spotify get songs didnt put playlist shuf...,1,1
5,2022-07-09 13:20:20,player controls sometimes disappear reason app...,3,7
6,2022-07-09 13:19:21,love selection lyrics provided song youre list...,5,0
7,2022-07-09 13:17:22,still extremely slow changing storage external...,3,16
8,2022-07-09 13:16:49,great app best mp3 music app ever used one pro...,5,0
9,2022-07-09 13:11:32,im deleting app following reasons app failing ...,1,318


# Exploratory Data Analysis

In [6]:
df['Rating'].value_counts()

5    22095
1    17653
4     7842
2     7118
3     6886
Name: Rating, dtype: int64